<div>
    <img style="float:right;" src="images/smi-logo.png"/>
    <div style="float:left;color:#58288C;"><h1>Datenanalyse und Datenmanagement</h1></div>
</div>

---
# Notebook III: Data Exploration
In diesem Notebook geht es um die Verwendung von Python zur Erkundung eines eingelesenen Datenbestands.  
Hierzu werden einfache Visualisierungsformen eingesetzt.

## Inhaltsverzeichnis

[1. Einführung in die Arbeit mit DataFrames am Beispiel von COVID-Daten](#kapitel1)  
[2. Einführung in die Datenvisualisierung mit Matplotlib](#kapitel2)  
[3. Übung / Anwendungsbeispiel Kreditvergabe](#kapitel3)  

---

In [ ]:
# Notebook extensions und Python Packages laden
%load_ext sql

import pandas as pd
import requests
import matplotlib.pyplot as plt

# Einen Farbstil für die zu erstellenden Diagramme auswählen
plt.style.use('seaborn-ticks')

# Jupyter-Feature: Wir wollen Diagramme direkt im Notebook sehen, nicht als Datei speichern o.ä.
%matplotlib inline

## 1. Einführung in die Arbeit mit DataFrames am Beispiel von COVID-Daten<a id="kapitel1"/>
Pandas sind das zentrale Werkzeug zum einlesen und manipulieren von Daten in Python. Für unsere Zwecke ist die Datenstruktur des ```DataFrames``` am wichtigsten:

> **DataFrame** is a 2-dimensional labeled data structure with columns of potentially different types.  
> You can think of it like a spreadsheet or SQL table [...]. It is generally the most commonly used pandas object.  
> [(Quelle)](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html)

Dataframes haben demnach Zeilen ("Rows") und Spalten ("Columns") können zudem einen Index haben - ein datensatzidentifizierendes Merkmal, ähnlich dem Primärschlüssel in der Datenbank.  
<img src="images/dataframe.png"/>  
DataFrames organisieren die Daten im Arbeitsspeicher, die mit SQL oder über APIs aus Datenquellen eingelesen werden. Die Dokumentation zu DataFrames findest du [hier](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html). 

Nutzen wir die Daten der COVID19-API aus der letzten Übung als Beispiel: 

In [ ]:
# API abfragen
response = requests.get("https://corona-api.com/countries/de")
df = pd.json_normalize(response.json()["data"]["timeline"])   # Im DataFrame df liegen nun die API-Daten

Nun sind die abgefragten Daten im DataFrame ```df```. Wir überprüfen zunächst oberflächlich die Fehlerfreiheit:

In [ ]:
df.head(5)   # die ersten fünf Zeilen anzeigen

In [ ]:
df.sample(5) # Zufällige fünf Zeilen anzeigen

In [ ]:
df.count()   # Anzahl der Werte je Spalte prüfen - gibt es Lücken?

### 1.2. Spalten auswählen und benennen

> Wichtig: Bei allen Bearbeitungen des DataFrames wird eine Kopie mit den Änderungen erzeugt. Das ursprüngliche DataFrame bleibt unverändert.  
> Soll eine Änderung vorgenommen werden, kann der Ursprungsvariablen das geänderte DataFrame direkt zugeordnet werden.  
> Beispiel: df = df.rename(...)

In [ ]:
# Bekommen wir aus der API auch unnötige Zeilen/Spalten, verwerfen wir diese zunächst
# (bei Datenbanken würden wir nur die benötigten Felder mit SQL abfragen)

# Die Funktion "drop" kann sowohl Zeilen als auch Spalten entfernen.
# Für Spalten ist eine Liste der Spaltennamen anzugeben, sowie der Verweis, dass es sich um Spalten handelt (axis="columns")

df = df.drop(["updated_at"], axis="columns")

# Dazu ist es oft praktisch, Spalten einheitlich oder kürzer zu benennen

df = df.rename(columns = {
    "new_confirmed": "new_cases", 
    "confirmed":"cases"
})

df.head(2)

In [ ]:
# Das Datum ist bislang eine reguläre Spalte, ein vom DataFrame separat geführter Index zählt die Zeilen von 0-n durch.
# Da es sich hier um Meldezeitpunkte handelt, ist das Datum der Index. Damit wird bspw. der Index automatisch als X-Achse in späteren Grafiken verwendet.

df = df.set_index("date")
df.head(2)

### 1.3. Daten sortieren und filtern

In [ ]:
# Sortieren
df = df.sort_values("date")

In [ ]:
df.head(5)

#### Filter
Der Zugriff auf eine Spalte ist mit ```df["feldname"]``` möglich, wenn der Name keine Leer- oder Sonderzeichen enthält auch vereinfacht mit ```df.feldname```.  

Mit ```df[filterkriterium]``` kann analog SELECT ... WHERE der Datensatz gefiltert werden.

In [ ]:
# Wir filtern nach Anzahl neuer Fälle und schreiben das Ergebnis in ein neues Dataframe

peaks = df[df.new_cases > 30000]                                       # alle Tage an denen > 30k Neuinfektionen auftraten

peaks = peaks.sort_values("new_cases", axis="rows", ascending=False)   # absteigend nach Anzahl der Neuinfektionen sortieren
peaks

### 1.4. DataFrames unterteilen
Mit DataFrames kann man ähnlich Listen und Dictionaries arbeiten - der Zugriff funktioniert nach dem Schema
> df.loc[Zeilenangabe, Spaltenangabe]  

Die Zeilen-/Spaltenangaben sind dabei Strings mit Index/Spaltenbezeichnungen, Listen oder Ranges.

In [ ]:
print("Zugriff Index und Spaltennamen mit .loc")
df.loc["2020-08-01","cases"]

In [ ]:
print("Ranges")
df.loc["2020-09-01":"2020-09-07", "deaths":"active"]

In [ ]:
print("Subsets")      # Listen von Zeilen und/oder Spalten
df.loc[["2020-02-01","2020-03-01"], 
             ["deaths", "active"]]

### 1.5. Berechnungen durchführen

In [ ]:
# Neue Spalte und Berechnungen

df["death_rate"] = df.new_deaths / df.active
df.death_rate                                   # der Index "date" wird automatisch mit angezeigt, wenn wir die neue Spalte abrufen

In [ ]:
# Deskriptive Statistiken können für das gesamte Dataframe oder einzelne Spalten berechnet werden

# df.describe()
df.new_cases.describe()

In [ ]:
# Für jede Spalte können mit analoger Syntax die wesentlichen statistischen Maße berechnet werden.
print("Median, Mean, Max, Min, 20% Quantil")
print(df.cases.median(), df.cases.mean(), df.cases.max(), df.cases.min(), df.cases.quantile(0.1))

---
## 2. Einführung in die Datenvisualisierung mit Matplotlib<a id="kapitel2"/>

### 2.1. Überblick
Pandas haben eine plot() Funktion, die einfache Grafiken erzeugt.

In [ ]:
df.new_cases.plot()

Mit Parametern kann das Aussehen der Diagramme konfiguriert werden. Die Dokumentation der ```plot```-Funktion findest du [hier](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.plot.html).

Nachfolgend ein Beispiel:

In [ ]:
df.new_cases.plot(kind="area", figsize=(18,5), color="darkblue", legend=True)             # figsize legt die Größe der Abbildung in Inches fest

### 2.2. Wichtige Charttypen

Über den Parameter "kind" können verschiedene Diagrammtypen erzeugt werden ([Dokumentation](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.plot.html)).

In [ ]:
# Histogramm der neuen Fälle

df.new_cases.plot(kind="hist", edgecolor="white") 

In [ ]:
# Ein Scatterplot zeigt zwei Merkmale auf X- und Y-Achse, der Aufruf der plot() Funktion ist daher etwas anders
df.plot(kind="scatter", x="new_cases", y="new_deaths", color="blue", title="Todesfälle vs. Neuinfektionen pro Tag")

### 2.3. Abbildung mehrerer Datenreihen
Sind zwei Plotting-Befehle in einer Notebook-Zelle, werden die entsprechenden Plots in einem Diagramm dargstellt.

In [ ]:
# Vergleich der neuen Fälle 2020/2021 mit Lineplots

df.loc[df.index >= "2021-01-01", "new_cases"].plot(kind="line", figsize=(16,5), alpha=0.5, color="blue", legend=True, label="Cases 2021") 
df.loc[df.index < "2021-01-01", "new_cases"].plot(kind="line", alpha=0.5, color="green", legend=True, label="Cases 2020") 

In [ ]:
# In manchen Fällen ist eine zweite Y-Achse erforderlich

df.new_cases.plot(kind="area", legend=True, figsize=(16,5))
df.death_rate.plot(legend=True, secondary_y=True)              # secondary_y trägt diese Datenreihe auf einer zweiten Y-Achse rechts auf

---
## 3. Übung / Anwendungsbeispiel Kreditvergabe <a id="kapitel3"/>

Wir betrachten im Folgenden einen Datensatz mit Privatkreditvorgängen einer deutschen Bank.  
Hierbei durchlaufen wir grundsätzlich dieselbe Prozedur.

Der Datensatz kommt vom Datebankserver und umfasst folgende Merkmalen:
   >  *Age* (numeric)  
   >  *Sex* (text: male, female)  
   >  *Job* (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)  
   >  *Housing* (text: own, rent, or free)  
   >  *Saving accounts* (text - little, moderate, quite rich, rich)  
   >  *Checking account* (text - little, moderate, quite rich, rich)   
   >  *Credit amount* (numeric, in EUR)  
   >  *Duration* (numeric, in month)  
   >  *Purpose* (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others  
   >  *Risk* (Value target - Good or Bad Risk)  

### 4.1. Daten einlesen und aufbereiten

Falls die SQLite-Datenbank noch nicht entpackt ist, dies nun tun:

In [ ]:
![ ! -f data/smi-data.db ] && unzip -o -d data data/smi-data.zip

In [ ]:
# Verbindung zum Datenbankserver herstellen

%load_ext sql
%sql sqlite:///data/smi-data.db

# SQL-Abfrage durchführen und Ergebnis in Variable result speichern        
        
result = %sql SELECT * FROM credit_ger

# Aus Result ein DataFrame machen und in Variable df speichern

df = result.DataFrame()
df = df.set_index(["id"])
# Spaltenbenennungen vereinfachen
df = df.rename({
    "Age": "age",
    "Sex": "sex",
    "Job": "job",
    "Housing": "housing",
    "Saving_accounts": "savings", 
    "Checking_account": "cash",
    "Credit_amount": "amount",
    "Duration": "duration",
    "Purpose": "purpose",
    "Risk": "risk"
}, axis="columns")

Sieh dir nun mit ```df.head()``` einige Datensätze für einen ersten Eindruck an!

### 4.2. Merkmale untersuchen und visualisieren

Erzeuge deskriptive Statistiken zu den Merkmalen age, duration und amount! Was lässt sich daraus über Kredite und Kreditnehmer im Datensatz sagen?

Untersuche mit Histogrammen die VMerkmale Alter (age), Kreditlaufzeit (duration) und Kreditsumme (amount) um einen ersten Eindruck zu erhalten.

Was ist die typische Kreditsumme, wie lange laufen Kredite meist und wie alt sind die Kreditnehmer?

Sieh dir nun in einem Histogramm die Kreditsummenverteilung von zurückgezahlen und ausgefallenen Krediten an (risk=="good" vs. risk=="bad")! Was ist festzustellen?

Die Funktion ```.value_counts()``` zählt textuelle Merkmalsausprägungen. Untersuche hiermit, welches die häufigsten Kreditzwecke (purpose) sind und erzeuge dazu ein Balkendiagramm (kind="bar" oder für horizontale Balken kind="barh")!

Erweitere dein voriges Balkendiagramm, indem du die Anzahlen der ausgefallenen Kredite je Zweck (purpose) in Rot darüberlegst.

### 4.3. Zusammenhänge untersuchen und visualisieren

Untersuche mit Scatterplots, ob sich zwischen amount, duration und age Zusammenhänge zeigen lassen! Was fällt auf?